In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
import h5py
import pickle
import os
import sys
cwd = os.path.abspath(os.getcwd())
import hoc_utils
import run_model
import config
os.chdir(config.neuron_path) 
from neuron import h
os.chdir(cwd)
print(os.getcwd())
import sys
import pandas as pd
import math

np.set_printoptions(threshold=sys.maxsize)



log_transform_params:  True
starting_pop_hack:  None
[b'34', b'11', b'39'] STIMS IN USE
found allen target volts


--No graphics will be displayed.


/pscratch/sd/z/zladd/axonstandardized/playground/runs/M1_TTPC_NA_HH_full_07_25_23_314831019/genetic_alg/neuron_genetic_alg_passive


In [10]:
model_num = str(config.model_num)
stims_path = f"../../stims/stims_{model_num}.hdf5"
target_volts_path = f'../../target_volts/allen_data_target_volts_{model_num}.hdf5'
opt_ind = config.params_opt_ind

# opt_ind = np.arange(7)

GA_result_path = '/pscratch/sd/z/zladd/axonstandardized/playground/runs/M1_TTPC_NA_HH_full_07_25_23_314831019/genetic_alg/neuron_genetic_alg_passive/best_indv_logs17393/best_indvs_gen_19.pkl'
params_path = '/pscratch/sd/z/zladd/axonstandardized/playground/runs/M1_TTPC_NA_HH_full_07_25_23_314831019/params/params_M1_TTPC_NA_HH_full.csv'


file_path_to_save = './plots/'
stim_ids_to_run = [ '11', '21', '34', '39']
# stim_ids_to_run = [ '34']

In [11]:
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

tick_major = 6
tick_minor = 4
plt.rcParams["xtick.major.size"] = tick_major
plt.rcParams["xtick.minor.size"] = tick_minor
plt.rcParams["ytick.major.size"] = tick_major
plt.rcParams["ytick.minor.size"] = tick_minor

font_small = 12
font_medium = 13
font_large = 14
plt.rc('font', size=font_small)          # controls default text sizes
plt.rc('axes', titlesize=font_medium)    # fontsize of the axes title
plt.rc('axes', labelsize=font_medium)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=font_small)    # fontsize of the tick labels
plt.rc('ytick', labelsize=font_small)    # fontsize of the tick labels
plt.rc('legend', fontsize=font_small)    # legend fontsize
plt.rc('figure', titlesize=font_large)   # fontsize of the figure title

# Plot for GA result
def final_indv_plot(param_names, final_best_indv, title, file_path_to_save=None, max_xtic=1, vert_size=10):
    plt.figure(figsize=(cm_to_in(8.5), cm_to_in(vert_size)))
    ax = plt.gca()
    y_pos = np.arange(len(param_names))
    ax.barh(y_pos, final_best_indv, height=0.5, align='center', linestyle='-', color='black')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(param_names)
    ax.invert_yaxis()  # labels read top-to-bottom
    plt.axvline(x=0, color='black', linewidth=0.4 ,linestyle='--')
    ax.set_xlim(0, max_xtic)
    ax.set_xticks([0, max_xtic])
    ax.set_ylabel('Parameters')
    ax.set_xlabel('Normalized Distance')
    ax.set_title('Deviation From Truth Value ' + title)
    if file_path_to_save:
        plt.savefig(file_path_to_save+'.pdf', format='pdf', dpi=1000, bbox_inches="tight")

# Code for optimization results analysis
def read_and_normalize_with_neg(opt_result_path, base, lower_bounds, upper_bounds):
    with open(opt_result_path, 'rb') as f:
        best_indvs = pickle.load(f, encoding = "latin1")
    normalized_indvs = []
    for i in range(len(best_indvs)):
        normalized = []
        for j in range(len(best_indvs[i])):
            if (best_indvs[i][j] < base[j]):
                new_value = abs((best_indvs[i][j] - base[j])/(ubs[j] - base[j]))
                normalized.append(new_value)
            else:
                new_value = abs((best_indvs[i][j] - base[j])/(ubs[j] - base[j]))
                normalized.append(new_value)
        normalized_indvs.append(normalized)
    return normalized_indvs, best_indvs

def plot_stim_volts_pair(stim, volts, title_stim, title_volts, file_path_to_save=None):
    fig = plt.figure(figsize=(cm_to_in(8.5+6), cm_to_in(12)))
    plt.subplot(2, 1, 1)
    plt.title(title_stim)
    plt.xlabel('Time (ms)')
    plt.ylabel('Amplitude (nA)')
    plt.plot(stim, color='black', linewidth=0.7)
    plt.subplot(2, 1, 2)
    plt.title('Voltage Response '+title_volts)
    plt.xlabel('Time (A.U.)')
    plt.ylabel('Amplitude (mV)')
    plt.plot(volts[0], label='target', color='black')
    plt.plot(volts[1], label='best individual', color='crimson')
    plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
    plt.tight_layout(pad=1)
    plt.show()
    if file_path_to_save:
        fig.savefig(file_path_to_save+'.pdf', format='pdf', dpi=1000, bbox_inches="tight")

def cm_to_in(cm):
    return cm/2.54

# Running a single volt
def run_single_volts(param_set, stim_data, ntimestep, dt):
    run_file = 'neuron_genetic_alg/neuron_files/compare_bbp/run_model_cori.hoc'
    h.load_file(run_file)
    total_params_num = len(param_set)
    timestamps = np.array([dt for i in range(ntimestep)])
    h.curr_stim = h.Vector().from_python(stim_data)
    h.transvec = h.Vector(total_params_num, 1).from_python(param_set)
    h.stimtime = h.Matrix(1, len(timestamps)).from_vector(h.Vector().from_python(timestamps))
    h.ntimestep = ntimestep
    h.runStim()
    out = h.vecOut.to_python()
    return np.array(out)

def plot_comb_scores(opt_path, score_path, title, plot_save_path=None):
    opt_result = h5py.File(opt_path)
    try:
        ordered_score_function_list = [e.decode('ascii') for e in opt_result['ordered_score_function_list'][:]]
        optimization_stim_names = [e.decode('ascii') for e in opt_result['opt_stim_name_list'][:]]
    except AttributeError:
        ordered_score_function_list = [e for e in opt_result['ordered_score_function_list'][:]]
        optimization_stim_names = [e for e in opt_result['opt_stim_name_list'][:]]
    optimization_weightes = opt_result['opt_weight_list'][:]
    best_stims_score_list = []
    for score_name in optimization_stim_names:
        curr_score_data = h5py.File(score_path+score_name+'_scores.hdf5', 'r')
        for sf in ordered_score_function_list:
            curr_stim_sf_pair = curr_score_data['norm_pin_scores_'+sf][:]
            best_stims_score_list.append(curr_stim_sf_pair)
    combined_score = sum([best_stims_score_list[i]*optimization_weightes[i] for i in range(len(optimization_weightes))])
    plt.figure(figsize=(cm_to_in(8.5), cm_to_in(5)))
    plt.title(title)
    plt.xlabel('Parameter Set Rank')
    plt.ylabel('Weighted Score')
    time_step = range(len(combined_score))
    plt.scatter(time_step, combined_score, s=1, color='black')
    if plot_save_path:
        plt.savefig(plot_save_path+'.pdf', format='pdf', dpi=1000, bbox_inches="tight")

        
def read_best_indvs(path, gen=-1):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data[gen]

def unlog_params(bases, best_full, config, opt_ind):
    if config.log_transform_params:
        for i in range(len(best_full)):
            if bases[i] > config.base_thresh and i in opt_ind:
                print('unlog : ', i)
                best_full[i] = math.pow(bases[i], best_full[i])
    return best_full


In [12]:
#read stims and target volts
stims = h5py.File(stims_path,"r")
target_volts = h5py.File(target_volts_path, 'r')

In [13]:
best_indv = read_best_indvs(GA_result_path)
df = pd.read_csv(params_path, skipinitialspace=True, usecols=['Param name','Base value', 'Lower bound', 'Upper bound'])
lbs = np.array(df['Lower bound'])
ubs = np.array(df['Upper bound'])
base_full = np.array(df['Base value'])
names = df['Param name']
bases, orig_params, mins, maxs = hoc_utils.log_params(ubs, lbs, base_full)

In [14]:
best_full = list(base_full)

for i, opt_idx  in enumerate(opt_ind):
    best_full[opt_idx] = best_indv[i]
    
best_full = unlog_params(bases, best_full, config, opt_ind)
best_full = np.abs(best_full)
best_full[config.negative_param_inds] = - best_full[config.negative_param_inds]


print("\n\n")
count = 0
for best, lb, ub, name in zip(best_full, lbs, ubs, names):
    if count in opt_ind:
        print(count, "(optimized)", "name :", name, " | best: ", round(best,9) , " | lb: ", round(lb,8), " | ub: ", round(ub,8))

    else:
        print(count, "name :", name, " | best: ", round(best,9) , " | lb: ", round(lb,8), " | ub: ", round(ub,8))
    print("------------------------------------------------------------")
    count += 1
    

unlog :  13
unlog :  15
unlog :  16
unlog :  17
unlog :  20



0 name : dend_na12  | best:  0.0005  | lb:  5e-05  | ub:  0.005
------------------------------------------------------------
1 name : soma_na12  | best:  0.4919775  | lb:  0.04919775  | ub:  4.919775
------------------------------------------------------------
2 name : ais_na12  | best:  5.0  | lb:  0.5  | ub:  10.0
------------------------------------------------------------
3 name : dend_na16  | best:  0.0005  | lb:  5e-05  | ub:  0.005
------------------------------------------------------------
4 name : soma_na16  | best:  0.4919775  | lb:  0.04919775  | ub:  4.919775
------------------------------------------------------------
5 name : ais_na16  | best:  7.0  | lb:  0.7  | ub:  70.0
------------------------------------------------------------
6 name : ais_ca  | best:  0.00099  | lb:  9.9e-05  | ub:  0.0099
------------------------------------------------------------
7 name : ais_KCa  | best:  0.007104  | lb:  0.0007104

In [16]:
    
for test_stim_name in stim_ids_to_run:
    if "dt" in test_stim_name:
        continue
    test_stim = stims[test_stim_name][:]
    test_stim2_dt = stims[test_stim_name + '_dt'][:]
    print(test_stim_name, test_stim2_dt[:])
    targV = target_volts[test_stim_name][:]
    response = run_model.run_model(best_full,test_stim_name,dt=test_stim2_dt, start_Vm=targV[0])
    response = response[0]
    file_save_path = f'./plots/{test_stim_name}'
    print('mean response: ', np.mean(response),' v init : ', targV[0] )
    # plot_stim_volts_pair(test_stim, [targV, response], test_stim_name+' (Testing)', 'for Full Parameters', file_save_path)
    fig = plt.figure(figsize=(15,8))
    plt.plot(targV, color='black', label='target')
    plt.plot(response, color='red', label='simulated')
    plt.legend()
    plt.ylabel('Vm (mV)')
    plt.xlabel('Timestep (A.U.)')
    fig.show()
    plt.savefig(f'plots/{test_stim_name}.png')
    plt.close()
#-59.61573578977873

11 [0.02]
running model at /pscratch/sd/z/zladd/axonstandardized/playground/runs/M1_TTPC_NA_HH_full_07_25_23_314831019/genetic_alg/neuron_genetic_alg_passive/neuron_files/M1_TTPC_NA_HH run dir is /pscratch/sd/z/zladd/axonstandardized/playground/runs/M1_TTPC_NA_HH_full_07_25_23_314831019/genetic_alg/neuron_genetic_alg_passive
ran working
ran working
mean response:  -75.58197619247922  v init :  -76.6875
21 [0.02]
running model at /pscratch/sd/z/zladd/axonstandardized/playground/runs/M1_TTPC_NA_HH_full_07_25_23_314831019/genetic_alg/neuron_genetic_alg_passive/neuron_files/M1_TTPC_NA_HH run dir is /pscratch/sd/z/zladd/axonstandardized/playground/runs/M1_TTPC_NA_HH_full_07_25_23_314831019/genetic_alg/neuron_genetic_alg_passive
ran working
ran working
mean response:  -75.38874401206772  v init :  -76.84375
34 [0.06]
running model at /pscratch/sd/z/zladd/axonstandardized/playground/runs/M1_TTPC_NA_HH_full_07_25_23_314831019/genetic_alg/neuron_genetic_alg_passive/neuron_files/M1_TTPC_NA_HH ru